# DNBLab Jupyter Notebook Tutorial 

## Titelanalyse Teil 2

In diesem Tutorial-Teil nutzen wir nun die vorher geschriebene Funktion, um bestimmte bibliographischen Informationen aus der MARC21-XML-Datei in ein Pandas Dataframe, oder alternativ eine Excel-Tabelle, zu überführen. Da wir später auch noch andere Fragestellungen mit Hilfe von Python an die Daten stellen möchten, legen wir das Dataframe zunächst für alle in der XML-Datei enthaltenen bibliographischen Einträge an.   

Zunächst importieren wir hierfür einen Teil unseres Codes aus dem vorherigen Tutorial: 

In [2]:
# Laden benötigter Python-Bibliotheken:
import xml.etree.ElementTree as ET
import pandas as pd

# Laden der MARC21-XML-Datei:
tree = ET.parse('dataset_tutorial_eco.xml')  # Laden der MARC-Datei in ElementTree 
root = tree.getroot()                             # Laden des Root-Verzeichnisses des XML

records = list(root)
total_records = len(records)  


# Ausgabe der Gesamtzahl der in der Datei vorhandenen Records:
print("Gesamtzahl bibliographischer Einträge: ",total_records)


# Funktion get_subfield_text:
def get_subfield_text(element, tag, subfield):
    if element.attrib['tag'] == tag:
            for subelement in element:
                if subelement.attrib['code'] == subfield:
                    return subelement.text
    return None
    

Gesamtzahl bibliographischer Einträge:  2828


Im folgenden überlegen wir, welche bibliographischen Angaben wir der Datei entnehmen wollen. Um im Sinne des Auffindes von Literatur alle wichtigen Angaben zu erhalten, brauchen wir Angaben zur Urheberschaft (Autor\*in etc.), Titel, Verlag, Verlagsort und Erscheinungsdatum. Wir arbeiten zwar bereits mit Daten, die sich auf Publikationen aus der Sachgruppe "Wirtschaft" beschränken, lassen uns zu Kontroll- und Übungszwecken aber auch diese mit ausgeben. Auch die DNB-eigene Identifiktationsnummer des Datensatzes werden wir zur besseren Überprüfbarkeit der Ergebnisse extrahieren. 

Die Liste der von uns gesuchten Daten sieht also folgendermaßen aus: 

* ID der DNB
* DNB-Sachgruppe
* Autor*in bzw. Herausgeber*in / Creator
* Titel / Title
* Verlag / Publisher
* Verlagsort / Place of publication
* Datum der Veröffentlichung / Date of publication


Um die Daten zu erhalten, schreiben wir uns eine Routine, die diese Angaben aus der XML-Datei holt und in ein Dataframe ablegt. Dafür legen wir die verschiedenen Elemente zunächst in für diese vorgesehene Listen ab und führen die Listen danach zu einem Dataframe zusammen. Die verschiedenen Angaben bzw. Elemente fragen wir dabei in der Reihenfolge ab, in der sie auch in der XML-Datei liegen: 


In [3]:
#Anlegen der zu befüllenden Listen: 
idcoll = []
ddccoll = []
authorcoll = []
titlecoll = []
placecoll = []
publishercoll = []
pubyear = []
interimlist = []     #Diese Liste wird nur zum Zwischenspeichern verwendet


for record in records:  
        
    for child in record.findall("{http://www.loc.gov/MARC21/slim}controlfield[@tag='001']"):   
        ident = child.text
        idcoll.append(ident)
       
    #Suche und Übernahme der Sachgruppe/DDC: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='082']"):
        for subelement in child:
            if subelement.attrib['code'] == "a":
                interimlist = []
                ddc = subelement.text
                interimlist.append(ddc)
            else:
                interimlist.append("Nicht vorhanden!")
                
    #Erstelle Liste mit jeweils nur dem ersten Element aus der jeweils vorangegangenen Interimsliste der Sachgruppen: 
    ddccoll.append(interimlist[0]) 
    
    
    #Suche und Übernahme von Autor*innen bzw. Herausgeber*innen: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='100']"):
        interimlist2 = []
        for subelement in child:
            if subelement.attrib['code'] == "a":
                mainauthor = subelement.text
                interimlist2.append(mainauthor)                
            elif not child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='100']"):  
                for child in record.find("{http://www.loc.gov/MARC21/slim}datafield[@tag='700']"): #finde erstes Kindelement "700"
                    for subelement in child:
                        if subelement.attrib['code'] == "a":
                            alternate = subelement.text
                            interimlist2.append(alternate)
                        else:
                            interimlist2.append("missing")
             
    authorcoll.append(interimlist2[0])
  
    #Suche und Übernahme des Titels: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield"):             
        title = get_subfield_text(child, "245", "a") 
        if title:
            titlecoll.append(title)
        
            
    
    #Suche und Übernahme des Publikationsjahres: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='264']"):   
        interimlist3 = []
        for subelement in child:
            if subelement.attrib['code'] == "c":
                rdapub = subelement.text
                interimlist3.append(rdapub)                
            elif not child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='264']"):  
                for child in record.find("{http://www.loc.gov/MARC21/slim}datafield[@tag='260']"): #finde erstes Kindelement "260"
                    for subelement in child:
                        if subelement.attrib['code'] == "c":
                            oldpub = subelement.text
                            interimlist3.append(oldpub)
                        else:
                            interimlist3.append("Nicht vorhanden")
             
    pubyear.append(interimlist3[0])

    
    #Suche und Übernahme des Publikationsortes: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='264']"):
        for subelement in child:
            if subelement.attrib['code'] == "a":
                interimlist = []
                place = subelement.text
                interimlist.append(place)
            else:
                interimlist.append("Nicht vorhanden!")
                 
    placecoll.append(interimlist[0]) 
    
    
    #Suche und Übernahme des Verlages: 
    for child in record.findall("{http://www.loc.gov/MARC21/slim}datafield[@tag='856']"):
        for subelement in child:
            if subelement.attrib['code'] == "u":
                interimlist = []
                publisher = subelement.text
                interimlist.append(publisher)
            else:
                interimlist.append("Nicht vorhanden!")
                
    publishercoll.append(interimlist[0]) 

    
#Anzeige der Anzahl der verschiedenen Elemente in den jeweiligen Listen:             
print("Elemente in ID-Collection:", len(idcoll))    
print("Elemente in DDC-Collection:", len(ddccoll))  
print("Elemente in Author-Collection:", len(authorcoll))
print("Elemente in Titel-Collection:", len(titlecoll))
print("Elemente in Publikationsjahr-Collection:", len(pubyear))
print("Elemente in Ort-Collection:", len(placecoll))
print("Elemente in Verlag-Collection:", len(publishercoll))
              

Elemente in ID-Collection: 2828
Elemente in DDC-Collection: 2828
Elemente in Author-Collection: 2828
Elemente in Titel-Collection: 2828
Elemente in Publikationsjahr-Collection: 2828
Elemente in Ort-Collection: 2828
Elemente in Verlag-Collection: 2828


Wir können hier überprüfen, dass wir in jeder Unterliste die selbe Anzahl an Einträgen vorfinden - diese sollte auch mit der Gesamtzahl an gefundenen bibliographischen Einträgen übereinstimmen. Ist dies der Fall, können wir die Listen im nächsten Schritt zu einer Tabelle zusammenführen und anschließend in ein Dataframe überführen: 


In [4]:
#Zusammenführen der Ergebnisse in eine Tabelle: 
result = {'Identifikator' : idcoll, 'Link' : publishercoll, 'Jahr' : pubyear}

#Erstellen eines Dataframes aus den Ergebnissen: 
df = pd.DataFrame(result)
#print(result)

#Gibt die ersten 5 Zeilen des Dataframes aus (zur Kontrolle):
df.head() 

#df.to_csv("AlleTitel.csv", index=False) 


,Identifikator,Link,Jahr
0,1043718966,https://d-nb.info/1043718966/34,2010
1,1019902175,http://eldiss.uni-kiel.de/macau/receive/disser...,2010
2,1012057232,http://opus.kobv.de/tuberlin/volltexte/2011/2898/,2010
3,1009567691,http://edoc.ub.uni-muenchen.de/12313/,2010
4,1045290823,http://publikationen.ub.uni-frankfurt.de/front...,2010


https://www.elab2go.de/demo-py1/jupyter-notebook-widgets.php

In [47]:
# Erstelle ein neues Text-Eingabefeld tb1 und zeige es an. 
tb1 = widgets.Text(value='Text eingeben', description='Text-Widget: ');display(tb1); 
# Erstelle IntSlider-Widget is1 und zeige es an. 
is1 = widgets.IntSlider(value=7,min=0, max=10, step=1, description='IntSlider: ');display(is1); 
# Erstelle Dropdown-Widget dd1 und zeige es an 
dd1 = widgets.Dropdown(options=['A', 'B', 'C'], value='A', description='Dropdown: ');display(dd1); 

Text(value='Text eingeben', description='Text-Widget: ')

IntSlider(value=7, description='IntSlider: ', max=10)

Dropdown(description='Dropdown: ', options=('A', 'B', 'C'), value='A')

In [56]:
# 3.1 Erstelle zwei Text-Eingabefelder 
textbox1 = widgets.Text(description='a');display(textbox1); 
textbox2 = widgets.Text(description='b');display(textbox2); 
# ... einen Button mit benutzerdefiniertem Layout 
button = widgets.Button(description='Berechnen!', layout=Layout(width='200px')); 
button.style.button_color = 'lightgreen';display(button); 
# ... und ein Text-Ausgabefeld für die Summe 
textbox3 = widgets.Text(description='Summe');display(textbox3); 
# 3.2 Definiere Eventhandler für den Button 
def on_button_clicked(sender): 
    a = int(textbox1.value); b = int(textbox2.value); 
    textbox3.value = str(a+b) 
# ... und weise ihn dem on_click-Ereignis zu 
button.on_click(on_button_clicked) 

Text(value='', description='a')

Text(value='', description='b')

Button(description='Berechnen!', layout=Layout(width='200px'), style=ButtonStyle(button_color='lightgreen'))

Text(value='', description='Summe')

In [29]:
# myfunc2 gibt den eingegebenen Text aus 
def myfunc2(x):  
     print('Ihre Eingabe: ' + str(x)) 
interact(myfunc2, x = 'Text eingeben'); 

interactive(children=(Text(value='Text eingeben', description='x'), Output()), _dom_classes=('widget-interact'…

In [26]:
!wget https://d-nb.info/1026552451/34

--2021-07-09 08:25:49--  https://d-nb.info/1026552451/34
Resolving d-nb.info (d-nb.info)... 193.175.100.223
Connecting to d-nb.info (d-nb.info)|193.175.100.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘34’

34                      [    <=>             ]   5.89M  6.97MB/s    in 0.8s    

2021-07-09 08:25:51 (6.97 MB/s) - ‘34’ saved [6179255]



In [27]:
!unzip 34

Archive:  34
  inflating: 06460010.tif            
  inflating: 06610000.tif            
  inflating: 1026552451_OCR.xml      
  inflating: 1026552451_OCR.txt      
  inflating: 06510015.tif            
  inflating: 06550019.tif            
  inflating: 0636000c.tif            
  inflating: 06540018.tif            
  inflating: 06500014.tif            
  inflating: 06620000.tif            
  inflating: 06570001.tif            
  inflating: 06470011.tif            
  inflating: 06440008.tif            
  inflating: 06580002.tif            
  inflating: 06560020.tif            
  inflating: 1026552451.pdf          
  inflating: 06400004.tif            
  inflating: 06370001.tif            
  inflating: 06420006.tif            
  inflating: 06450009.tif            
  inflating: 06590003.tif            
  inflating: 06480012.tif            
  inflating: 06600004.tif            
  inflating: 06410005.tif            
  inflating: 06390003.tif            
  inflating: 06490013.tif            

Wir können uns auch die Gesamtlänge des Dataframes folgendermaßen anzeigen lassen: 

In [6]:
rows_dataframe = df[df.columns[0]].count()
print("Gesamtzahl an Einträgen:", rows_dataframe)

Gesamtzahl an Einträgen: 2828


Da wir uns vorher bereits die Anzahl der jeweiligen Elemente pro Variable haben anzeigen lassen, ist das Ergebnis in diesem Fall wenig überraschend und dient vor allem dem Abgleich.

Mit diesem Dataframe können wir nun auf verschiedenste Arten weiterarbeiten - weitere Tutorials folgen! 

In [8]:
input = open ('1026552451_OCR.txt','r')
output = open ('Ergebnisse.txt','w')
x = input.read()

def neu():
    i = 0
    while i < len(x):
        word = x[i:i+8]
        if word == 'AMERIKA':
            string= x[i+9:i+15]
            print(string)
        i += 1

In [2]:
datei = open('1026552451_OCR.txt','r')
print(datei)

<_io.TextIOWrapper name='1026552451_OCR.txt' mode='r' encoding='UTF-8'>


In [28]:
input = open ('1026552451_OCR.txt','r')
output = open ('Ergebnisse.txt','w')
x = input.read()
whitespace = ' '

def neu():
    i = 0
    while i < len(x):
        word = x[i:i+6]
        if word == 'LA':
            string= x[i+8:i+19]
            print(string)
            output.write(string)
            output.write(whitespace)
        i += 1
    input.close()
    output.close()

In [14]:
import os
import sys
import fileinput

print ("Text to search for:")
textToSearch = input( "AMERIKA" ) 

print ("Text to replace it with:")
textToReplace = input( "AMERIKAII" )

print ("File to perform Search-Replace on:")
fileToSearch  = input( "> " )
#fileToSearch = 'D:\dummy1.txt'

tempFile = open( fileToSearch, 'r+' )

for line in fileinput.input( fileToSearch ):
    if textToSearch in line :
        print('Match Found')
    else:
        print('Match Not Found!!')
    tempFile.write( line.replace( textToSearch, textToReplace ) )
tempFile.close()


input( '\n\n Press Enter to exit...' )

Text to search for:


TypeError: '_io.TextIOWrapper' object is not callable

In [30]:
file = open("1026552451_OCR.txt", "w")

def check_string():
    with open('1026552451_OCR.txt') as temp_f:
        datafile = temp_f.readlines()
    for line in datafile:
        if 'LA' in line:
            return True # The string is found
    return False  # The string does not exist in the file

if check_string():
    print('True')
else:
    print('False')

False
